# Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.



In [2]:
import pandas as pd

df = pd.read_csv('ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [22]:
# решение с pandas
uid_grouped = df.groupby('userId')
udf = pd.DataFrame()
udf['recs_num'] = uid_grouped.count()['movieId']
udf['user_ttl'] = uid_grouped.timestamp.max()-uid_grouped.timestamp.min()

udf = udf[udf.recs_num >= 100]

udf.head()

,recs_num,user_ttl
userId,,
4,204,203560
5,100,2101
8,116,85187
15,1700,471393496
17,363,8053


In [23]:
!head ratings.csv -n3

In [29]:
# альтернативный вариант (тема занятия была всё же про альтернативные варианты обработки больших массивов)
# проверим что записи отсортированы по uid
uid_sorted = True
res = {}
with open('ratings.csv') as f:
    hdr = f.readline()
    
    p_uid = None
    min_ts, max_ts = None, None
    recs_num = 0
    for ln in f:
        uid, _, _, ts = ln.strip().split(',')
        uid, ts = int(uid), int(ts)   
        
        if p_uid is None or p_uid < uid:
            # новый uid пошёл
            if recs_num >= 100:
                # запишем ttl для юзера
                res[p_uid] = max_ts - min_ts            
            p_uid, min_ts, max_ts = uid, ts, ts            
            recs_num = 1
        elif uid < p_uid:
            # uid может(должен) быть только больше или равен предыдущему значению
            uid_sorted = False
            print('dataset not sorted by uid')
            break
        else:
            # запись для уже известного юзера
            recs_num = recs_num + 1
            min_ts = min([ts, min_ts])
            max_ts = max([ts, max_ts])
        
if uid_sorted:
    for uid in res:
        print(f'lifetime of user {uid} is {res[uid]} seconds')

lifetime of user 4 is 203560 seconds
lifetime of user 5 is 2101 seconds
lifetime of user 8 is 85187 seconds
lifetime of user 15 is 471393496 seconds
lifetime of user 17 is 8053 seconds
lifetime of user 19 is 5282 seconds
lifetime of user 21 is 1365432 seconds
lifetime of user 22 is 91491 seconds
lifetime of user 23 is 18342129 seconds
lifetime of user 26 is 20267261 seconds
lifetime of user 30 is 115852276 seconds
lifetime of user 33 is 4332211 seconds
lifetime of user 34 is 2835 seconds
lifetime of user 36 is 5949290 seconds
lifetime of user 38 is 59971665 seconds
lifetime of user 41 is 15930495 seconds
lifetime of user 43 is 13885 seconds
lifetime of user 48 is 136315977 seconds
lifetime of user 56 is 8564120 seconds
lifetime of user 57 is 4323 seconds
lifetime of user 61 is 1374418 seconds
lifetime of user 67 is 4136 seconds
lifetime of user 68 is 152083904 seconds
lifetime of user 72 is 2945466 seconds
lifetime of user 73 is 220585493 seconds
lifetime of user 75 is 22560475 seconds

# Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.



Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [41]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
display(rzd)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
display(auto)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
display(air)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
display(client_base)

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [42]:
for t in [rzd, auto, air, client_base]:
    t.set_index('client_id', inplace=True)

In [47]:
# сформировать таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
tbl_1 = rzd.join([air, auto], how='outer').fillna(0).reset_index()
display(tbl_1.head())

,client_id,rzd_revenue,air_revenue,auto_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,0.0,57483.0
3,114,5774.0,0.0,83.0
4,115,981.0,81.0,912.0


In [51]:
# проверки по суммам
display(tbl_1.sum())
display(air.sum())
display(auto.sum())
display(rzd.sum())
# всё ок

client_id         916.0
rzd_revenue     20941.0
air_revenue       271.0
auto_revenue    63410.0
dtype: float64

air_revenue    271
dtype: int64

auto_revenue    63410
dtype: int64

rzd_revenue    20941
dtype: int64

In [53]:
# всё тоже самое (вроде), но вместо client_id делаем адрес клиента
tbl_2_columns = 'address rzd_revenue air_revenue auto_revenue'.split(' ')
tbl_2 = tbl_1.set_index('client_id').join(client_base, how='outer').fillna(0).reset_index()[tbl_2_columns]
tbl_2

,address,rzd_revenue,air_revenue,auto_revenue
0,Комсомольская 4,1093.0,0.0,0.0
1,Энтузиастов 8а,2810.0,0.0,0.0
2,Левобережная 1а,10283.0,0.0,57483.0
3,Мира 14,5774.0,0.0,83.0
4,ЗЖБИиДК 1,981.0,81.0,912.0
5,Строителей 18,0.0,4.0,4834.0
6,Панфиловская 33,0.0,13.0,98.0
7,Мастеркова 4,0.0,173.0,0.0


# Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы.  
**Как бы вы добавили эти сведения в таблицу визитов и покупок?**  
Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

# вариант 1
Допущения:
- координаты также дополнены меткой времени
- все записи в исходной таблице также имеют метку времени

Ответ:
1. В таблицу визитов и покупок добавляем столбцы: user_geo_latitude, user_geo_lontitude, user_geo_city (string или city_id), user_geo_address (string).
2. Задаём max_time_diff в секундах. Эта переменная определяет насколько метка времени о координатах пользователях может отличаться от метки времени записи в визитах/покупках
3. Заполняем для каждого пользователя столбцы п. 1 с учетом max_time_diff, выбирая минимально удаленное во времени местоположение.
4. Дополнительно можно добавить столбцы по аналогии prev_user_geo (предыдущее расположение пользователя) + prev_user_geo_ts (временная метка предыдущего расположения).

## вариант 2
Допущения:
- координаты без каких либо временных меток
- считаем что координаты последовательны

Ответ:
1. Дополняем таблицу истории местоположений пользователя колонками типа country, city, address
2. Для каждого пользователя определяем его наиболее частую локацию и добавляем информацию по ней в таблицу покупок и визитов, но просто как информацию о пользователе.

Однозначно связать географию пользователя с записями визитов/покупок без возможности связать, например, по времени, эту информацию - проблематично. Нужно больше информации о контексте задачи, данных...

### Вариант 3
Допущения:
- в координатах пользователя вообще нет ничего, кроме (user_id, [geopoints]), даже по времени не упорядочены

Что можно посчитать:
- определить для пользователя bounding box его расположения;
- наиболее частую координату или область(области) с наибольшей плотностью точек;
- ...

И все эти извлеченные "признаки" дописать в истоирю визитов/покупок в удобном для последующего использования виде.

### Ответы на доп вопросы
> У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

Варианты:
- есть временные метки у записей гео и истории визитов/покупок - связываем по времени и userid. Возможные проблемы. 
- у пользователя было открыто несколько сессий одновременно в разных местах (или из одного устройства шли координаты, а с другого были действия). Тогда данные должны быть дополнены чем-то вроде: идентификатор сессии, ip-откуда был визит/покупка (как минимум нерелевантную географию можем отбросить), ip откуда приехала география по пользователю и т.п. 

IP-пользовательских запросов, идентификатор сессии, идентификатор устройства и т.п. логировать - технический вопрос, поэтому как минимум на метку времени полагаться можно всегда. 

> Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

Зависит от того, что и как получиться мапить в таблицу визитов/покупок.
В худшем случае - просто каша и город (страну) пользователя определим. 
По идее ещё "характер" сервиса влияет, для которого эти данные собраны. Например, если это "сервис путешественников" - то разброс координат у пользователей может быть огромный и это нормально, а если это магазин детских расходных товаров или кухонных товаров - то типовой пользователь скорее всего будет привязан к дому/городу.


> Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Варианты:
- Пропустить через геокодер и дополнить координаты страной, городом, адресами.
- Если есть временные метки и богатая история по пользователю - определить где он живёт (вечерние координаты - дом, дневные - работа/универ, и т.п.)
- Определить дополнительную информацию к профилю пользователя - где бывает (торговые центры/места отдыха), ездит ли за пределы города/страны, наличие машины/пользуется ли часто такси или ездит общественным транспортом/метро. 
- пользуется ли средствами анонимизации - география с устройства отличается от geoip, откуда приходят данные. Если пользуется - можно маркировать для более пристального внимания.

